In [ ]:
import os
from pathlib import Path
import torch
import torch.nn as nn

import onnx
from onnx_tf.backend import prepare
import tensorflow as tf

from dataloader import make_loader
# from model import DepthwiseSeparable

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:



class DepthwiseSeparable(nn.Module):
    def __init__(self, c_in, c_out):
        super().__init__()
        self.depthwiseConv = nn.Conv2d(c_in, c_in, kernel_size=3, padding=1, groups=c_in, bias=False)
        self.pointwiseConv = nn.Conv2d(c_in, c_out, kernel_size=1, bias=False)
        self.b_norm = nn.BatchNorm2d(c_out)
        self.relu = nn.ReLU(inplace=True)
    def forward(self, x):
        x = self.depthwiseConv(x)
        x = self.pointwiseConv(x)
        x = self.b_norm(x)
        return self.relu(x)
    
class TinyVAD(nn.Module):
    def __init__(self, n_mels=40):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1, bias=False),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True),
        )
        self.b1 = DepthwiseSeparable(16, 24)
        self.b2 = DepthwiseSeparable(24, 32)
        self.b3 = DepthwiseSeparable(32, 48)
        self.head = nn.Conv2d(48, 1, kernel_size=1)  # [B,1,M,T]

        self.n_mels = n_mels

    def forward(self, x):  # x: [B, 1, n_mels(=40), T]
        x = self.stem(x)
        x = self.b1(x) 
        x = self.b2(x) 
        x = self.b3(x)  # [B,48,M,T]
        x = self.head(x)                                 # [B, 1, M, T]
        x = x.mean(dim=2)                                # 沿 mel 平均 -> [B, 1, T]
        return x.squeeze(1)                              # [B, T]

test_ds, test_dl = make_loader(split="test", batch_size=1, shuffle=False, num_workers=0)

# 取得 T_fixed
with torch.no_grad():
    sample = next(iter(test_dl))
    feat = sample["feat"]   # [1, 40, T]
    T_fixed = feat.shape[-1]
print(f"T_fixed = {T_fixed}")

# 載入 checkpoint
ckpt_dir = Path("./checkpoints")
ckpt_path = ckpt_dir / "best.pt"
assert ckpt_path.exists(), f"No checkpoint：{ckpt_path}"

state = torch.load(ckpt_path, map_location="cpu")

model = TinyVAD(n_mels=40).to(device)
model.load_state_dict(state["model"], strict=True)
model.eval()

# PyTorch weight
weights_only_path = Path("tinyvad_state_dict_only.pt")
torch.save(model.state_dict(), weights_only_path)
print(f"weights-only PyTorch file: {weights_only_path.resolve()}")


c:\Users\TUF\myenv\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


#train = 16000, #valid = 2000, #test = 2000
feat shape: torch.Size([4, 40, 400])
label shape: torch.Size([4, 400])
paths[0]: D:\VAD_project\data\training_data\train\result\009147
sample 0: T=400, VAD+ ratio=0.538
sample 1: T=400, VAD+ ratio=0.495
sample 2: T=400, VAD+ ratio=0.530
sample 3: T=400, VAD+ ratio=0.535
device: cuda
T_fixed = 400
weights-only PyTorch file: D:\VAD_project\VAD_model\tinyvad_state_dict_only.pt


In [ ]:
# ONNX
onnx_dynamic_path = Path("tinyvad_dynamic.onnx")
onnx_fixed_path   = Path("tinyvad_fixed.onnx")

# Dummy input
dummy_fixed = torch.randn(1, 1, 40, T_fixed, device=device)  # 固定 T
dummy_dynamic = dummy_fixed.clone()

# Dynamic ONNX
torch.onnx.export(
    model, dummy_dynamic, onnx_dynamic_path.as_posix(),
    input_names=["input"], output_names=["logits"],
    dynamic_axes={"input": {0: "batch", 3: "time"},
                  "logits": {0: "batch", 1: "time"}},
    opset_version=11, do_constant_folding=True
)
print(f"Export dynamic ONNX: {onnx_dynamic_path.resolve()}")

# Fixed ONNX
torch.onnx.export(
    model, dummy_fixed, onnx_fixed_path.as_posix(),
    input_names=["input"], output_names=["logits"],
    dynamic_axes=None,  # 全固定： [1,1,40,T_fixed] -> [1,T_fixed]
    opset_version=11, do_constant_folding=True
)
print(f"Export fixed-shape ONNX: {onnx_fixed_path.resolve()}")


Export dynamic ONNX: D:\VAD_project\VAD_model\tinyvad_dynamic.onnx
Export fixed-shape ONNX: D:\VAD_project\VAD_model\tinyvad_fixed.onnx


#### representative dataset

In [ ]:
T_FIXED = 400
LAYOUT = "NCHW"

def representative_dataset(dataloader, max_windows=800):
    import numpy as np
    cnt = 0
    for batch in dataloader:
        feats = batch["feat"]            # 形狀 [B, 40, T]
        for i in range(feats.shape[0]):
            M = feats[i].cpu().numpy()   # [40, T]
            #print("M shape: ", M.shape) # [40, 400]
            
            # 對齊 T
            if M.shape[1] > T_FIXED:
                M = M[:, -T_FIXED:]
            elif M.shape[1] < T_FIXED:
                M = np.pad(M, ((0,0),(0,T_FIXED - M.shape[1])), mode="constant")

            # 模型吃的 layout
            if LAYOUT == "NCHW":
                X = M[np.newaxis, np.newaxis, :, :]
            else:
                X = M[np.newaxis, :, :, np.newaxis]
            X = X.astype("float32")

            yield [X]
            cnt += 1
            if cnt >= max_windows:
                return

train_ds, train_dl = make_loader(split="train", batch_size=8, shuffle=False, num_workers=0)
def represent_data_gen():
    yield from representative_dataset(train_dl, max_windows=800)


In [ ]:
# ONNX -> TensorFlow SavedModel -> TFLite 

onnx_path = Path("tinyvad_fixed.onnx")  # Use Fixed ONNX, 減少 TFLite 形狀問題
assert onnx_path.exists(), f"No ONNX ：{onnx_path}"

# 1) ONNX -> TensorFlow SavedModel
onnx_model = onnx.load(onnx_path.as_posix())
tf_rep = prepare(onnx_model)
saved_model_dir = Path("tf_savedmodel_tinyvad")
# 若資料夾存在就覆蓋
if saved_model_dir.exists():
    import shutil; shutil.rmtree(saved_model_dir)
tf_rep.export_graph(saved_model_dir.as_posix())
print(f"Exported TensorFlow SavedModel to: {saved_model_dir.resolve()}")

# 2) TensorFlow SavedModel -> TFLite (fp32 baseline)
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir.as_posix())
tflite_fp32 = converter.convert()
tflite_fp32_path = Path("tinyvad_fp32.tflite")
with open(tflite_fp32_path, "wb") as f:
    f.write(tflite_fp32)
print(f"Exported TFLite (float32) to: {tflite_fp32_path.resolve()}")

# 3) TFLite fp16
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir.as_posix())
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_fp16 = converter.convert()
tflite_fp16_path = Path("tinyvad_fp16.tflite")
with open(tflite_fp16_path, "wb") as f:
    f.write(tflite_fp16)
print(f"Exported TFLite (float16) to: {tflite_fp16_path.resolve()}")

# 4) INT8
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir.as_posix())
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = represent_data_gen  #representative dataset

# 用 int8但不改 I/O 型別
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
tflite_int8 = converter.convert()
tflite_int8_path = Path("tinyvad_int8.tflite")
with open(tflite_int8_path, "wb") as f:
    f.write(tflite_int8)
print(f"Exported TFLite (int8) to: {tflite_int8_path.resolve()}")



INFO:tensorflow:Assets written to: tf_savedmodel_tinyvad\assets


INFO:tensorflow:Assets written to: tf_savedmodel_tinyvad\assets


Exported TensorFlow SavedModel to: D:\VAD_project\VAD_model\tf_savedmodel_tinyvad
Exported TFLite (float32) to: D:\VAD_project\VAD_model\tinyvad_fp32.tflite
Exported TFLite (float16) to: D:\VAD_project\VAD_model\tinyvad_fp16.tflite
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:  (40, 400)
M shape:

In [5]:
# Compare
def human_size(p: Path):
    n = p.stat().st_size
    units = ["B","KB","MB","GB"]
    i = 0
    while n >= 1024 and i < len(units)-1:
        n /= 1024
        i += 1
    return f"{n:.2f} {units[i]}"

files = []

# Original checkpoint
ckpt_dir = Path("./checkpoints")
for name in ["best.pt", "last.pt"]:
    p = ckpt_dir / name
    if p.exists():
        files.append(("PyTorch checkpoint", p.name, p.resolve(), human_size(p)))
        break  # 只列一個

# weight（only state_dict）
p = Path("tinyvad_state_dict_only.pt")
if p.exists():
    files.append(("PyTorch weights-only", p.name, p.resolve(), human_size(p)))

# ONNX
for name in ["tinyvad_dynamic.onnx", "tinyvad_fixed.onnx"]:
    p = Path(name)
    if p.exists():
        files.append(("ONNX", p.name, p.resolve(), human_size(p)))

# TFLite
for name in ["tinyvad_fp32.tflite", "tinyvad_fp16.tflite", "tinyvad_int8.tflite"]:
    p = Path(name)
    if p.exists():
        if "fp32" in name:
            tag = "TFLite (fp32)"
        elif "fp16" in name:
            tag = "TFLite (fp16)"
        elif "int8" in name:
            tag = "TFLite (int8)"
        else:
            tag = "TFLite"
        files.append((tag, p.name, p.resolve(), human_size(p)))

for kind, name, full, size in files:
    print(f"{kind:>18}: {name:25s} | {size:>10s} | {full}")


PyTorch checkpoint: best.pt                   |   65.42 KB | D:\VAD_project\VAD_model\checkpoints\best.pt
PyTorch weights-only: tinyvad_state_dict_only.pt |   25.45 KB | D:\VAD_project\VAD_model\tinyvad_state_dict_only.pt
              ONNX: tinyvad_dynamic.onnx      |   16.59 KB | D:\VAD_project\VAD_model\tinyvad_dynamic.onnx
              ONNX: tinyvad_fixed.onnx        |   16.57 KB | D:\VAD_project\VAD_model\tinyvad_fixed.onnx
     TFLite (fp32): tinyvad_fp32.tflite       |   20.30 KB | D:\VAD_project\VAD_model\tinyvad_fp32.tflite
     TFLite (fp16): tinyvad_fp16.tflite       |   14.84 KB | D:\VAD_project\VAD_model\tinyvad_fp16.tflite
     TFLite (int8): tinyvad_int8.tflite       |   15.93 KB | D:\VAD_project\VAD_model\tinyvad_int8.tflite
